In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,66620.42,-0.90,-0.17,-0.08,0.00,-0.01,0.0,-0.00
3,-0.90,8418.29,0.06,-0.04,0.01,-0.00,-0.0,0.00
4,-0.17,0.06,2890.47,-0.01,0.02,0.01,-0.0,-0.00
5,-0.08,-0.04,-0.01,655.46,-0.00,-0.00,0.0,0.00
6,0.00,0.01,0.02,-0.00,136.96,0.00,0.0,0.00
7,-0.01,-0.00,0.01,-0.00,0.00,24.82,-0.0,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,3.6,0.00
9,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,0.32


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00001,0.00000,-0.00001,0.00000,-0.00001
3,-0.00004,1.00000,0.00001,-0.00002,0.00001,-0.00001,-0.00001,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00004,0.00004,-0.00000,-0.00002
5,-0.00001,-0.00002,-0.00001,1.00000,-0.00001,-0.00001,0.00001,0.00000
6,0.00000,0.00001,0.00004,-0.00001,1.00000,0.00002,0.00000,0.00003
7,-0.00001,-0.00001,0.00004,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00000,-0.00001,-0.00000,0.00001,0.00000,-0.00001,1.00000,0.00002
9,-0.00001,0.00000,-0.00002,0.00000,0.00003,-0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.3019164215989056

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.54994210e+08 8.70840557e+06 9.93232245e+05 4.91293689e+04
 2.01171375e+03 5.87402558e+01 9.27674638e-01 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999812,-0.018344,-0.006195,-0.001402,-0.000293,-0.000053,-0.000008,-7.114383e-07
1,0.017887,0.997672,-0.064513,-0.012721,-0.002638,-0.000479,-0.000069,-6.174068e-06
2,0.007273,0.063675,0.996713,-0.048592,-0.009542,-0.001723,-0.000251,-2.256385e-05
3,0.001959,0.015573,0.047064,0.997211,-0.054945,-0.009429,-0.001364,-1.219300e-04
4,0.000511,0.004042,0.011784,0.053679,0.996472,-0.062697,-0.008705,-7.749441e-04
5,0.000123,0.000975,0.002836,0.012534,0.061255,0.995258,-0.074179,-6.416041e-03
6,0.000027,0.000212,0.000617,0.002719,0.013021,0.072613,0.993235,-8.963348e-02
7,0.000005,0.000038,0.000111,0.000488,0.002335,0.012897,0.088904,9.959538e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00018849014246202778,
 0.002328136145751114,
 0.0032867220434716904,
 0.0027894525352774036,
 0.003528308505058253,
 0.00474220632223421,
 0.0067649393494806365,
 0.004046156491046227]